In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cleanee/IQR_data.csv


In [2]:
dftrain = pd.read_csv('/kaggle/input/cleanee/IQR_data.csv')

In [3]:
# Street: object, 
#     City: object, 
#         County: object, 
#             State: object, 
#                 Airport_Code: object, 
#                     Wind_Direction: object, 
#                         Weather_Condition: object, 
#                             Sunrise_Sunset: object

In [4]:
cat_cols = ['Street','City','County','State','Airport_Code','Wind_Direction','Weather_Condition','Sunrise_Sunset']

# Convert categorical columns to category type
for col in cat_cols:
    dftrain[col] = dftrain[col].astype('category')

In [5]:
for cols in dftrain.columns:
    print(cols, len(dftrain[cols].unique()))

Unnamed: 0 243019
Year 3
Severity 4
Start_Lat 134988
Start_Lng 134347
Distance(mi) 3674
Street 42361
City 6857
County 1277
State 49
Airport_Code 1616
Temperature(F) 271
Wind_Chill(F) 573
Visibility(mi) 49
Wind_Direction 23
Weather_Condition 70
Traffic_Signal 2
Sunrise_Sunset 2
TimeDiff 3822


In [6]:
df1 = dftrain.drop(columns=['Unnamed: 0'])

In [7]:
df_ = df1.drop(columns=['Severity'])

In [8]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import preprocessing
from collections import Counter

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [9]:
X = df1.drop(columns=['Severity'])
y = df1['Severity']
y = y-np.ones(y.shape[0])

In [10]:
y,df1['Severity']

(0         1.0
 1         2.0
 2         1.0
 3         2.0
 4         1.0
          ... 
 243014    1.0
 243015    1.0
 243016    3.0
 243017    3.0
 243018    3.0
 Name: Severity, Length: 243019, dtype: float64,
 0         2
 1         3
 2         2
 3         3
 4         2
          ..
 243014    2
 243015    2
 243016    4
 243017    4
 243018    4
 Name: Severity, Length: 243019, dtype: int64)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y , random_state=42,test_size=0.25, shuffle=False)

In [12]:
# def f_importance_plot(f_imp):
#     fig = plt.figure(figsize=(12, 0.20*len(f_imp)))
#     plt.title('Feature importances', size=16, y=1.05, 
#               fontweight='bold', color='#444444')
#     a = sns.barplot(data=f_imp, x='avg_imp', y='feature', 
#                     palette='YlOrBr_r', linestyle="-", 
#                     linewidth=0.5, edgecolor="black")
#     plt.xlabel('')
#     plt.xticks([])
#     plt.ylabel('')
#     plt.yticks(size=11, color='#444444')
    
#     for j in ['right', 'top', 'bottom']:
#         a.spines[j].set_visible(False)
#     for j in ['left']:
#         a.spines[j].set_linewidth(0.5)
#     plt.tight_layout()
#     plt.show()
# def show_confusion_roc(oof: list) -> None:
#     """Draws a confusion matrix and roc_curve with AUC score.
        
#         Args:
#             oof: predictions for each fold stacked. (list of tuples)
        
#         Returns:
#             None
#     """
    
#     f, ax = plt.subplots(1, 2, figsize=(13.3, 4))
#     df = pd.DataFrame(np.concatenate(oof), columns=['id', 'preds', 'target']).set_index('id')
#     df.index = df.index.astype(int)
#     cm = confusion_matrix(df.target, df.preds.ge(0.5).astype(int))
#     cm_display = ConfusionMatrixDisplay(cm).plot(cmap='YlOrBr_r', ax=ax[0])
#     ax[0].grid(False)
#     RocCurveDisplay.from_predictions(df.target, df.preds, color='#20BEFF', ax=ax[1])
#     plt.tight_layout();
    
# def get_mean_auc(oof_results):
#     """Calculate the mean AUC from out-of-fold predictions."""
#     true_values = []
#     preds = []
    
#     for oof in oof_results:
#         true_values.extend(oof[:, 2])
#         preds.extend(oof[:, 1])
    
#     return roc_auc_score(true_values, preds,multi_class='ovr')

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import re as re
from collections import Counter

from tqdm.auto import tqdm
import math
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay
import warnings
warnings.filterwarnings('ignore')

import time
from xgboost import XGBClassifier
%matplotlib inline
tqdm.pandas()

# rc = {
#     "axes.facecolor": "#FFF9ED",
#     "figure.facecolor": "#FFF9ED",
#     "axes.edgecolor": "#000000",
#     "grid.color": "#EBEBE7",
#     "font.family": "serif",
#     "axes.labelcolor": "#000000",
#     "xtick.color": "#000000",
#     "ytick.color": "#000000",
#     "grid.alpha": 0.4
# }

# sns.set(rc=rc)

# from colorama import Style, Fore
# red = Style.BRIGHT + Fore.RED
# blu = Style.BRIGHT + Fore.BLUE
# mgt = Style.BRIGHT + Fore.MAGENTA
# gld = Style.BRIGHT + Fore.YELLOW
# res = Style.RESET_ALL
# bold_start = Style.BRIGHT
# bold_end = Style.NORMAL

In [14]:
from sklearn.metrics import accuracy_score

In [15]:
FOLDS = 20
SEED = 1004
xgb_models = []
xgb_oof = []
# predictions = np.zeros(len(dftest))
f_imp = []
xgb_valAccu=[]
xgb_TrainAccu=[]

counter = 1
X = df1.drop(columns=['Severity'])
y = df1['Severity'] - np.ones(df1['Severity'].shape[0])
skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y)):
    if (fold + 1) % 5 == 0 or (fold + 1) == 1:
        print(f'{"#" * 24} Training FOLD {fold + 1} {"#" * 24}')
    
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]
    watchlist = [(X_train, y_train), (X_valid, y_valid)]

    # XGboost model and fit with GPU support
    model = XGBClassifier(n_estimators=1000, n_jobs=-1, max_depth=3, eta=0.2, colsample_bytree=0.67, tree_method='gpu_hist',enable_categorical=True,verbosity=0)
    model.fit(X_train, y_train, eval_set=watchlist, early_stopping_rounds=300,verbose=False)

    val_preds = model.predict_proba(X_valid)
    train_preds = model.predict_proba(X_train)
    val_pred = model.predict(X_valid)
#     val_accu = accuracy_score(y_valid, val_preds)
#     print(val_accu)
    val_score = roc_auc_score(y_valid, val_preds,multi_class='ovr')
    train_score = roc_auc_score(y_train, train_preds,multi_class='ovr')
    best_iter = model.best_iteration
    xgb_valAccu.append(val_score)
    xgb_TrainAccu.append(train_score)
#     idx_pred_target = np.vstack([val_idx, val_preds, y_valid]).T  # shape(len(val_idx), 3)
#     f_imp.append({i: j for i, j in zip(X.columns, model.feature_importances_)})
    print(f'{" " * 20}AUC: {val_score:.5f} {" " * 6}Best Iteration: {best_iter}')

#     xgb_oof.append(idx_pred_target)
#     xgb_models.append(model)
#     if val_score > 0.80:
#         test_preds = model.predict_proba(dftest.drop(columns=['id']))[:, 1]
#         predictions += test_preds
#         counter += 1

# predictions /= counter
# mean_val_auc = get_mean_auc(np.array(xgb_oof))
# print(f'{bold_start}{red}{"*" * 45}{res}')
# print(f'Mean AUC: {red}{mean_val_auc:.5f}{res}{bold_end}')

######################## Training FOLD 1 ########################
                    AUC: 0.85997       Best Iteration: 999
                    AUC: 0.90847       Best Iteration: 997
                    AUC: 0.95178       Best Iteration: 998
                    AUC: 0.92197       Best Iteration: 997
######################## Training FOLD 5 ########################
                    AUC: 0.93671       Best Iteration: 997
                    AUC: 0.88099       Best Iteration: 998
                    AUC: 0.89270       Best Iteration: 999
                    AUC: 0.82246       Best Iteration: 997
                    AUC: 0.90670       Best Iteration: 998
######################## Training FOLD 10 ########################
                    AUC: 0.94223       Best Iteration: 999
                    AUC: 0.90144       Best Iteration: 999
                    AUC: 0.85566       Best Iteration: 999
                    AUC: 0.85088       Best Iteration: 999
                    AUC: 0.87524  

In [16]:
print("Mean of Training score")
print(np.mean(xgb_TrainAccu))


Mean of Training score
0.9888844225788759


In [17]:
print("Mean of Validation score")
print(np.mean(xgb_valAccu))


Mean of Validation score
0.8975710489254778


In [18]:
df1.shape

(243019, 18)

In [19]:
df1['Severity'].shape[0]

243019